In [70]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset
# torch 本身的套件
import torch
# torch functions
import torch.nn.functional as F
# torch neural network functions
import torch.nn as nn
# optim = optimizer
import torch.optim as optim
# easier for data optimisation
import torch.utils.data as Data
# 可以簡單看到自己的 nn model 疊成甚麼鳥樣
from torchinfo import summary
# 就...numpy
import numpy as np
# 畫圖用 plotting
import matplotlib.pyplot as plt
DEVICE = torch.device("cuda")
# 確認自己是用 cpu 還是 gpu (gpu就會寫CUDA)
# 為了以後方便請查一下怎麼變CUDA，如果你是cpu的話。
import torchvision
import torchvision.transforms as transforms
print(DEVICE)

cuda


In [71]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

'''
data augmentation
transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
'''

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [72]:
print(trainset[0][0].shape)
#getting the shape for each data
print(len(trainset))
#train set size

torch.Size([3, 32, 32])
50000


In [73]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 就是把一群東西堆成一個block
        self.cnn = nn.Sequential(
            # conv1
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),

            # conv2
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),

            # conv3
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),

            # conv4
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),

            # # conv5
            # nn.Conv2d(512, 512, 3, padding=1),
            # nn.ReLU(),
            # nn.Conv2d(512, 512, 3, padding=1),
            # nn.ReLU(),
            # nn.Conv2d(512, 512, 3, padding=1),
            # nn.ReLU(),
            # nn.MaxPool2d(2, stride=2)
        )
        self.fc = nn.Sequential(
            nn.Linear(128*4*4, 520),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(520, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = x.view([x.size()[0], -1])
        x = self.fc(x)
        return x


net = Net().cuda()
summary(net, input_size=(16, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [16, 10]                  --
├─Sequential: 1-1                        [16, 512, 2, 2]           --
│    └─Conv2d: 2-1                       [16, 64, 32, 32]          1,792
│    └─ReLU: 2-2                         [16, 64, 32, 32]          --
│    └─Conv2d: 2-3                       [16, 64, 32, 32]          36,928
│    └─ReLU: 2-4                         [16, 64, 32, 32]          --
│    └─MaxPool2d: 2-5                    [16, 64, 16, 16]          --
│    └─Conv2d: 2-6                       [16, 128, 16, 16]         73,856
│    └─ReLU: 2-7                         [16, 128, 16, 16]         --
│    └─Conv2d: 2-8                       [16, 128, 16, 16]         147,584
│    └─ReLU: 2-9                         [16, 128, 16, 16]         --
│    └─MaxPool2d: 2-10                   [16, 128, 8, 8]           --
│    └─Conv2d: 2-11                      [16, 256, 8, 8]           29

In [74]:
optimizer = optim.Adam(net.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [75]:
def run_epoch(dataloader, update=True):
    total_num, total_hit, total_loss = 0, 0, 0
    for now_step, batch_data in enumerate(dataloader):
        # 將優化器歸零 (需要更新的參數會放在optimizer裡面，所以下一次更新要手動清0)
        optimizer.zero_grad()
        # 拿這次要訓練的 batch
        inputs, labels = batch_data
        # 將資料轉到 device (有GPU就是GPU)
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        # 將資料過NN
        if update:
            logits = net(inputs)
            loss = criterion(logits, labels) 
            loss.backward()
            # 將記錄在優化器的參數更新到模型上面
            optimizer.step()    
        else:
            # 宣告自己沒有要更新model的意思
            with torch.no_grad():
                logits = net(inputs)
                loss = criterion(logits, labels) 
            
        # 計算有沒有算對
        total_hit += torch.sum(torch.argmax(logits, dim=1) == labels).item()

        total_num += len(inputs)
        total_loss += loss.item() * len(inputs)
    return total_loss / total_num, total_hit / total_num


In [76]:
now_best_acc = 0
for epoch in range(20):
    net.train()
    train_loss, train_acc = run_epoch(trainloader, update=True)
    net.eval()
    valid_loss, valid_acc = run_epoch(testloader, update=False)

    print('epoch {:>3d}: train loss: {:6.4f}, acc {:6.4f} valid loss: {:6.4f}, acc {:6.4f}'.format(
        epoch, train_loss, train_acc, valid_loss, valid_acc))
